In [ ]:
# @title Install Requirment
!pip install git+https://github.com/epg900/whisper.git
!pip install -U deep-translator
!pip install rich

clear_output()
import whisper
model = whisper.load_model("turbo")
from IPython.display import clear_output
clear_output()
#!whisper vid.mp4 --model medium --language it

In [ ]:
# @title Upload Video and Download Subtitle
#results = model.transcribe("vid.mp4", language="it", task="translate")
from deep_translator import GoogleTranslator
from rich.progress import track
from google.colab import files

uploaded = files.upload()
clear_output()

result = model.transcribe(next(iter(uploaded)))

class dotdict(dict):
  def __getattr__(self,key):
     return self[key]
  def __setattr__(self,key,value):
    self[key] = value

def format_time(seconds_float):
    total_milliseconds = round(seconds_float * 1000)
    hours, remainder = divmod(abs(total_milliseconds), 3600000)
    minutes, remainder = divmod(remainder, 60000)
    seconds, milliseconds = divmod(remainder, 1000)

    sign = '-' if seconds_float < 0 else ''
    return f"{sign}{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

f = open(f"{next(iter(uploaded)).split('.')[0]}.srt",'w',encoding='utf-8')
f2 = open(f"{next(iter(uploaded)).split('.')[0]}_origlang.srt",'w',encoding='utf-8')
for k in track(result['segments'], description="Processing..."):
  fatxt = GoogleTranslator(source='auto', target='fa').translate(k['text'])
  f.write(f"{k['id']}\n{format_time(k['start'])} --> {format_time(k['end'])}\n{fatxt}\n\n")
  f2.write(f"{k['id']}\n{format_time(k['start'])} --> {format_time(k['end'])}\n{k['text']}\n\n")
f.close()
f2.close()
files.download(f"{next(iter(uploaded)).split('.')[0]}.srt")
files.download(f"{next(iter(uploaded)).split('.')[0]}_origlang.srt")